# data process

In [1]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import numpy as np
import os
import glob

Using TensorFlow backend.


In [2]:
class dataProcess(object):
    def __init__(self, out_rows, out_cols, aug_merge_path="./data/aug_merge", aug_train_path="./data/aug_train",
                 aug_label_path="./data/aug_label", test_path = './data/test', npy_path="./data/npydata",
                 img_type="tif"):
        self.out_rows = out_rows
        self.out_cols = out_cols
        self.aug_merge_path = aug_merge_path
        self.aug_train_path = aug_train_path
        self.aug_label_path = aug_label_path
        self.test_path = test_path
        self.npy_path = npy_path
        self.img_type = img_type

    def create_train_data(self):
        # 将增强之后的训练集生成npy         
        i = 0
        print('-' * 30)
        print('creating train image')
        print('-' * 30)
        count = 0
        #计算增强后样本数目
        for indir in os.listdir(self.aug_merge_path):
            path = os.path.join(self.aug_merge_path, indir)
            count += len(os.listdir(path))
            
        imgdatas = np.ndarray((count, self.out_rows, self.out_cols, 1), dtype=np.uint8)
        imglabels = np.ndarray((count, self.out_rows, self.out_cols, 1), dtype=np.uint8)
        
        for indir in os.listdir(self.aug_merge_path):
            trainPath = os.path.join(self.aug_train_path, indir)
            labelPath = os.path.join(self.aug_label_path, indir)
            print(trainPath, labelPath)
            imgs = glob.glob(trainPath + '/*' + '.tif')
            for imgname in imgs:
                trainmidname = imgname[imgname.rindex('/') + 1:]
                labelimgname = imgname[imgname.rindex('/') + 1:imgname.rindex('_')] + '_label.tif'
                print(trainmidname, labelimgname)
                img = load_img(trainPath + '/' + trainmidname, grayscale=True)
                label = load_img(labelPath + '/' + labelimgname, grayscale=True)
                img = img_to_array(img)
                label = img_to_array(label)
                imgdatas[i] = img
                imglabels[i] = label
                if i % 100 == 0:
                    print('Done: {0}/{1} images'.format(i, len(imgs)))
                i += 1
                print(i)
        print('loading done', imgdatas.shape)
        np.save(self.npy_path + '/imgs_train.npy', imgdatas)            # 将30张训练集和30张label生成npy数据
        np.save(self.npy_path + '/imgs_mask_train.npy', imglabels)
        print('Saving to .npy files done.')
        
    def create_test_data(self):

        # 测试集生成npy
        i = 0
        print('-' * 30)
        print('Creating test images...')
        print('-' * 30)
        imgs = glob.glob(self.test_path + "/*." + self.img_type)           # deform/train
        print(len(imgs))
        imgdatas = np.ndarray((len(imgs), self.out_rows, self.out_cols, 1), dtype=np.uint8)
        for imgname in imgs:
            midname = imgname[imgname.rindex("/") + 1:]   # 图像的名字
            img = load_img(self.test_path + "/" + midname, grayscale=True)   # 转换为灰度图
            img = img_to_array(img)
            imgdatas[i] = img
            if i % 100 == 0:
                print('Done: {0}/{1} images'.format(i, len(imgs)))
            i += 1
        print('loading done', imgdatas.shape)
        np.save(self.npy_path + '/imgs_test.npy', imgdatas)            # 将30张训练集和30张label生成npy数据
        # np.save(self.npy_path + '/imgs_mask_train.npy', imglabels)
        print('Saving to .npy files done.')
        
    def load_train_data(self):
        # 读入训练数据包括label_mask(npy格式), 归一化(只减去了均值)
        print('-' * 30)
        print('load train images...')
        print('-' * 30)
        imgs_train = np.load(self.npy_path + "/imgs_train.npy")
        imgs_mask_train = np.load(self.npy_path + "/imgs_mask_train.npy")
        imgs_train = imgs_train.astype('float32')
        imgs_mask_train = imgs_mask_train.astype('float32')
        imgs_train /= 255
        mean = imgs_train.mean(axis=0)
        imgs_train -= mean
        imgs_mask_train /= 255
        imgs_mask_train[imgs_mask_train > 0.5] = 1
        imgs_mask_train[imgs_mask_train <= 0.5] = 0
        return imgs_train, imgs_mask_train
    
    def load_test_data(self):
        print('-' * 30)
        print('load test images...')
        print('-' * 30)
        imgs_test = np.load(self.npy_path + "/imgs_test.npy")
        imgs_test = imgs_test.astype('float32')
        imgs_test /= 255
        mean = imgs_test.mean(axis=0)
        imgs_test -= mean
        return imgs_test

In [3]:
mydata = dataProcess(512, 512)
mydata.create_train_data()
mydata.create_test_data()
imgs_train, imgs_mask_train = mydata.load_train_data()
print(imgs_train.shape, imgs_mask_train.shape)

------------------------------
creating train image
------------------------------
./data/aug_train/25 ./data/aug_label/25
25_0_6851_train.tif 25_0_6851_label.tif
Done: 0/31 images
1
25_0_5270_train.tif 25_0_5270_label.tif
2
25_0_7788_train.tif 25_0_7788_label.tif
3
25_0_5136_train.tif 25_0_5136_label.tif
4
25_0_9644_train.tif 25_0_9644_label.tif
5
25_0_966_train.tif 25_0_966_label.tif
6
25_0_514_train.tif 25_0_514_label.tif
7
25_0_2711_train.tif 25_0_2711_label.tif
8
25_0_2220_train.tif 25_0_2220_label.tif
9
25_0_9688_train.tif 25_0_9688_label.tif
10
25_0_9911_train.tif 25_0_9911_label.tif
11
25_0_3183_train.tif 25_0_3183_label.tif
12
25_0_9723_train.tif 25_0_9723_label.tif
13
25_0_3140_train.tif 25_0_3140_label.tif
14
25_0_8889_train.tif 25_0_8889_label.tif
15
25_0_8301_train.tif 25_0_8301_label.tif
16
25_0_9303_train.tif 25_0_9303_label.tif
17
25_0_5420_train.tif 25_0_5420_label.tif
18
25_0_1208_train.tif 25_0_1208_label.tif
19
25_0_7791_train.tif 25_0_7791_label.tif
20
25_0_8445_tr

186
./data/aug_train/21 ./data/aug_label/21
21_0_5321_train.tif 21_0_5321_label.tif
187
21_0_8223_train.tif 21_0_8223_label.tif
188
21_0_4950_train.tif 21_0_4950_label.tif
189
21_0_5991_train.tif 21_0_5991_label.tif
190
21_0_7142_train.tif 21_0_7142_label.tif
191
21_0_3702_train.tif 21_0_3702_label.tif
192
21_0_8905_train.tif 21_0_8905_label.tif
193
21_0_3558_train.tif 21_0_3558_label.tif
194
21_0_8183_train.tif 21_0_8183_label.tif
195
21_0_1685_train.tif 21_0_1685_label.tif
196
21_0_5853_train.tif 21_0_5853_label.tif
197
21_0_186_train.tif 21_0_186_label.tif
198
21_0_5897_train.tif 21_0_5897_label.tif
199
21_0_5790_train.tif 21_0_5790_label.tif
200
21_0_8203_train.tif 21_0_8203_label.tif
Done: 200/31 images
201
21_0_5369_train.tif 21_0_5369_label.tif
202
21_0_8146_train.tif 21_0_8146_label.tif
203
21_0_3646_train.tif 21_0_3646_label.tif
204
21_0_2959_train.tif 21_0_2959_label.tif
205
21_0_9381_train.tif 21_0_9381_label.tif
206
21_0_8712_train.tif 21_0_8712_label.tif
207
21_0_1829_trai

371
18_0_3405_train.tif 18_0_3405_label.tif
372
./data/aug_train/23 ./data/aug_label/23
23_0_303_train.tif 23_0_303_label.tif
373
23_0_6162_train.tif 23_0_6162_label.tif
374
23_0_6368_train.tif 23_0_6368_label.tif
375
23_0_381_train.tif 23_0_381_label.tif
376
23_0_5040_train.tif 23_0_5040_label.tif
377
23_0_3238_train.tif 23_0_3238_label.tif
378
23_0_6098_train.tif 23_0_6098_label.tif
379
23_0_5179_train.tif 23_0_5179_label.tif
380
23_0_9083_train.tif 23_0_9083_label.tif
381
23_0_6072_train.tif 23_0_6072_label.tif
382
23_0_1748_train.tif 23_0_1748_label.tif
383
23_0_5754_train.tif 23_0_5754_label.tif
384
23_0_7649_train.tif 23_0_7649_label.tif
385
23_0_397_train.tif 23_0_397_label.tif
386
23_0_1146_train.tif 23_0_1146_label.tif
387
23_0_2290_train.tif 23_0_2290_label.tif
388
23_0_3661_train.tif 23_0_3661_label.tif
389
23_0_9593_train.tif 23_0_9593_label.tif
390
23_0_3188_train.tif 23_0_3188_label.tif
391
23_0_4722_train.tif 23_0_4722_label.tif
392
23_0_1311_train.tif 23_0_1311_label.ti

559
27_0_4553_train.tif 27_0_4553_label.tif
560
27_0_3415_train.tif 27_0_3415_label.tif
561
27_0_2814_train.tif 27_0_2814_label.tif
562
27_0_1265_train.tif 27_0_1265_label.tif
563
27_0_9925_train.tif 27_0_9925_label.tif
564
27_0_1796_train.tif 27_0_1796_label.tif
565
27_0_1189_train.tif 27_0_1189_label.tif
566
27_0_7196_train.tif 27_0_7196_label.tif
567
27_0_1005_train.tif 27_0_1005_label.tif
568
27_0_8736_train.tif 27_0_8736_label.tif
569
27_0_2869_train.tif 27_0_2869_label.tif
570
27_0_845_train.tif 27_0_845_label.tif
571
27_0_1462_train.tif 27_0_1462_label.tif
572
27_0_9150_train.tif 27_0_9150_label.tif
573
27_0_3027_train.tif 27_0_3027_label.tif
574
27_0_6151_train.tif 27_0_6151_label.tif
575
27_0_6777_train.tif 27_0_6777_label.tif
576
27_0_4708_train.tif 27_0_4708_label.tif
577
27_0_2648_train.tif 27_0_2648_label.tif
578
27_0_1916_train.tif 27_0_1916_label.tif
579
27_0_5068_train.tif 27_0_5068_label.tif
580
27_0_8562_train.tif 27_0_8562_label.tif
581
27_0_7817_train.tif 27_0_7817_

751
24_0_9380_train.tif 24_0_9380_label.tif
752
24_0_4614_train.tif 24_0_4614_label.tif
753
24_0_3900_train.tif 24_0_3900_label.tif
754
24_0_7201_train.tif 24_0_7201_label.tif
755
24_0_7648_train.tif 24_0_7648_label.tif
756
24_0_9759_train.tif 24_0_9759_label.tif
757
24_0_8034_train.tif 24_0_8034_label.tif
758
24_0_9955_train.tif 24_0_9955_label.tif
759
24_0_5713_train.tif 24_0_5713_label.tif
760
24_0_7034_train.tif 24_0_7034_label.tif
761
24_0_7166_train.tif 24_0_7166_label.tif
762
24_0_2663_train.tif 24_0_2663_label.tif
763
24_0_4278_train.tif 24_0_4278_label.tif
764
24_0_8459_train.tif 24_0_8459_label.tif
765
24_0_4937_train.tif 24_0_4937_label.tif
766
24_0_3920_train.tif 24_0_3920_label.tif
767
24_0_3402_train.tif 24_0_3402_label.tif
768
24_0_2880_train.tif 24_0_2880_label.tif
769
24_0_992_train.tif 24_0_992_label.tif
770
24_0_7187_train.tif 24_0_7187_label.tif
771
24_0_8582_train.tif 24_0_8582_label.tif
772
24_0_3788_train.tif 24_0_3788_label.tif
773
24_0_4757_train.tif 24_0_4757_

(930, 512, 512, 1) (930, 512, 512, 1)


# unet

In [4]:
from keras.models import *
from keras.layers import Input, merge, Conv2D, MaxPooling2D, UpSampling2D, Dropout, Cropping2D, BatchNormalization
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler

In [6]:
class myUnet(object):
    def __init__(self, img_rows=512, img_cols=512):
        self.img_rows = img_rows
        self.img_cols = img_cols

    def load_data(self):
        mydata = dataProcess(self.img_rows, self.img_cols)
        imgs_train, imgs_mask_train = mydata.load_train_data()
        imgs_test = mydata.load_test_data()
        return imgs_train, imgs_mask_train, imgs_test
    
    def get_unet(self):
        inputs = Input((self.img_rows, self.img_cols, 1))

        conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
        conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
        pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

        conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
        conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
        pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

        conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool2)
        conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
        pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

        conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool3)
        conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv4)
        drop4 = Dropout(0.5)(conv4)
        pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

        conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool4)
        conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv5)
        drop5 = Dropout(0.5)(conv5)

        up6 = Conv2D(512, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
            UpSampling2D(size=(2, 2))(drop5))
        merge6 = merge([drop4, up6], mode='concat', concat_axis=3)
        conv6 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge6)
        conv6 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv6)


        up7 = Conv2D(256, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
            UpSampling2D(size=(2, 2))(conv6))
        merge7 = merge([conv3, up7], mode='concat', concat_axis=3)
        conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge7)
        conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv7)

        up8 = Conv2D(128, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
            UpSampling2D(size=(2, 2))(conv7))
        merge8 = merge([conv2, up8], mode='concat', concat_axis=3)
        conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge8)
        conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv8)

        up9 = Conv2D(64, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
            UpSampling2D(size=(2, 2))(conv8))
        merge9 = merge([conv1, up9], mode='concat', concat_axis=3)
        conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge9)
        conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)
        conv9 = Conv2D(2, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)
        conv10 = Conv2D(1, 1, activation='sigmoid')(conv9)

        model = Model(input=inputs, output=conv10)

        model.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy', metrics=['accuracy'])
        print('model compile')
        return model
    
    def train(self):
        print("loading data")
        imgs_train, imgs_mask_train, imgs_test = self.load_data()
        print("loading data done")
        model = self.get_unet()
        print("got unet")

        # 保存的是模型和权重,
        model_checkpoint = ModelCheckpoint('unet.hdf5', monitor='loss', verbose=1, save_best_only=True)
        print('Fitting model...')
        model.fit(imgs_train, imgs_mask_train, batch_size=1, epochs=10, verbose=1, shuffle=True,
                  callbacks=[model_checkpoint])

        print('predict test data')
        imgs_mask_test = model.predict(imgs_test, batch_size=1, verbose=1)
        np.save('imgs_mask_test.npy', imgs_mask_test)
        
    
    def test(self):
        print("loading data")
        imgs_train, imgs_mask_train, imgs_test = self.load_data()
        print("loading data done")
        model = self.get_unet()
        print("got unet")
        model.load_weights('unet.hdf5')
        print('predict test data')
        imgs_mask_test = model.predict(imgs_test, batch_size=1, verbose=1)
        np.save('imgs_mask_test.npy', imgs_mask_test)

In [ ]:
myunet = myUnet()
myunet.train()

loading data
------------------------------
load train images...
------------------------------
------------------------------
load test images...
------------------------------
loading data done


/home/ldy/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:38: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/home/ldy/anaconda3/envs/tensorflow/lib/python3.6/site-packages/keras/legacy/layers.py:465: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/home/ldy/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:45: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/home/ldy/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:51: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `con

model compile
got unet
Fitting model...
Epoch 1/10
  6/930 [..............................] - ETA: 28:25:30 - loss: 0.6937 - acc: 0.6930